In [1]:
# python core
import time
import os
import pickle
import itertools as it

#numpy/pandas
import numpy as np
import pandas as pd

#language modeling: Gensim, sentencepiece
from gensim.models import Doc2Vec
from gensim.models.callbacks import CallbackAny2Vec
from gensim import utils

import sentencepiece as spm

#Scikit learn
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import OneHotEncoder

#Tensorflow
from tensorflow.keras.layers import Lambda, Input, Dense, MaxPooling1D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras import utils
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

#Bio python
from Bio import SeqIO

#Ipython
from IPython.display import clear_output

In [2]:
#drop samples without labels, drop any samples with no otus (shouldnt be any but just in case)
#drop otus that arent in any samples 
def clean_table(study_name):
    print(study_name,":")
    df = pd.read_table('microbiome_datasets/selected_datasets_joined/{}.tsv'.format(study_name),sep=',',low_memory=False)
    print("Shape before cleaning: ", df.shape)
    #Drop samples with no label.
    drop = df.isna().any()
    drop[["OTU_ID", 'Sequence']] = False #Dont drop the sequence and OTU columns. Sequence has NaN for disease state.
    df = df.loc[:, ~drop]
    #drop rows with all zeros. There shouldnt be any but just in case.
    df = df.loc[~(df[df.columns]==0).all(axis=1)]
    #Drop columns with all zeros. again, there shouldnt be any but just in case.
    df = df.loc[:, (df!=0).any(axis=0)]
    print("Shape after cleaning: ", df.shape)
    return df

    

In [3]:
#Convert sequence to kmers seperated by white space
def seq_to_kmers(seq, k):
    index = 0
    out = ""
    while index < len(seq) - k:
        out += "{0} ".format(seq[index: index + k])
        index += 1
    out += seq[index:index+k]
    return out

#Used during Doc2Vec training, must be defined here to surpress error.
class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.time = None
    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
        self.time = time.time()
    def on_epoch_end(self, model):
        e = time.time() - self.time
        print("Epoch #{0} end in {1}".format(self.epoch, e))
        self.epoch += 1

### Data set processing methods
Note: Compute time could be saved by combining these methods. Keeping htem seperate, however, allows for individual analyses using any given method as well as comparison of run times

In [4]:
#Qiime2 uses a simple norm so i'll do that for now.
def relative_abundance_xy(df):
    samples = df[[col for col in df.columns if col not in ['OTU_ID', 'Sequence']]].T.values
    X = samples[:,:-1]
    y = samples[:,-1]
    
    X = X.astype(np.float)
    X = X / X.sum()
    # return relative abundance matrix and y labels
    return X, y

"""
df: the standard otu dataframe
sp: sentencepiece model
d2v: doc2vec model

#encode all the sequences
# for each sample, multiply 
"""
def bpe_docs_xy(df, sp, d2v):
    print("Encoding tokens...")
    bpe_encoded = [sp.encode_as_pieces(seq) for seq in df['Sequence'].values[:-1]]
    print("Encoding vectors...")
    as_vectors = [d2v.infer_vector(seq) for seq in bpe_encoded]
    print("Processing...")
    samples = [col for col in df.columns if col not in ['OTU_ID', "Sequence"]]
    X = []
    y = []
    for sample in samples:
        vecs = []
        values = df[sample].values[:-1].astype(np.float)
        values = values / values.sum()
        for i, value in enumerate(values):
            weighted_vec = value * as_vectors[i]
            vecs.append(weighted_vec)
        X.append(vecs)
        y.append(df[sample].values[-1])
    X = np.asarray(X).sum(axis=1)
    print("Done.")
    return X, y
 
def kmer_docs_xy(df, d2v, k):
    print("Formatting sequences...")
    kmers = [seq_to_kmers(seq, k).split(' ') for seq in df['Sequence'].values[:-1]]
    print("Encoding vectors...")
    as_vectors = [d2v.infer_vector(seq) for seq in kmers]
    print("Processing...")
    samples = [col for col in df.columns if col not in ['OTU_ID', "Sequence"]]
    X = []
    y = []
    for sample in samples:
        vecs = []
        values = df[sample].values[:-1].astype(np.float)
        values = values / values.sum()
        for i, value in enumerate(values):
            weighted_vec = value * as_vectors[i]
            vecs.append(weighted_vec)
        X.append(vecs)
        y.append(df[sample].values[-1])
    X = np.asarray(X).sum(axis=1)
    print("Done.")
    return X, y


### Machine learning methods

In [11]:
def labels_to_onehot(labels):
    labels = np.asarray(labels).reshape(-1,1)
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(labels)
    return ohe.transform(labels)

def prepare_for_keras(X, y):
    return np.reshape(X, (X.shape[0], X.shape[1], 1)), labels_to_onehot(y)


# Create a simple CNN model.
# Returns a single-layer 1D CNN with pooling and dropout.
# Input param k is the embedding dimension.
# TODO maybe allow param adjustment?
def create_model(emb_dim):
    inputs = layers.Input(shape=(emb_dim,1,))
    x = layers.Conv1D(32, 2)(inputs)
    x = layers.MaxPooling1D()(x)
    x = layers.Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(30, activation='relu')(x)
    outputs = layers.Dense(2,activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)

    model.summary()
    model.compile(loss='binary_crossentropy',
                   optimizer='Adam', metrics=['accuracy'])
    return model



"""
Method for adding noisy artificial data to the data set.
i.e. gaussian noise data augmentaiton

-- Params --
x_data: The training data with labels removed
y: The labels corresponding to x data
magnitude: The number of fold of x_data to synthesize.
           If magnitude == 1, len(x_data) will be syntheized.
           If magnitude == 2, 2 * len(x_data) will be synthesized, etc.
sigma: Standard deviation for gaussian distribution of noise centered around 0.
emb)dim: cardinality of embedding dimension

"""

def gaussian_expansion(x_data,y, magnitude, sigma, emb_dim):
    mu = 0.0
    num_samples = x_data.shape[0]
    x_noised = []
    for i in range(magnitude):
        noise = np.random.normal(mu, sigma, x_data.shape)
        x_noised.append(x_data + noise)

    y_new = [y] * (magnitude +1)
    y_new = np.asarray(y_new).reshape(y.shape[0]*(magnitude+1), y.shape[1])
    x_noised = np.asarray(x_noised)
    x_noised = x_noised.reshape(magnitude*x_data.shape[0],emb_dim,1)
    #y_new = enc.transform(np.reshape(y_new, (-1,1)))
    new_x = np.concatenate((x_data, x_noised))

    return(new_x, y_new)
    


"""
Method for running k-fold CV with neural network model.

-- Params --
X: The entire dataset, no labels.
y: The labels.
n_folds: Number of folds CV
sigma: std dev for gaussian noise
noise_size: number fold to increase data set with noise
enc: sklearn OneHotEncoder object
desc: Description dictionary to provide readable results
emb_dim: number of features selected
"""
def kfold_cnn(X, y, 
              n_folds, 
              sigma_scale, noise_size, 
              emb_dim, 
              dataset_name,
              processing_name,
              otu_table=False):
    skf = StratifiedKFold(n_splits=n_folds)
    scores = []
    y = np.asarray(y)
    
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(y.reshape(-1,1))
    
    #In each train/test split, split the train into train/validation.
    for train, test in skf.split(X, y):
        print(dataset_name, processing_name)
        
        X_new = X
        #if its a count table, need to select K best. 
        if otu_table:
            print("Reducing OTU table dimension...")
            kbest = SelectKBest(mutual_info_classif, k=emb_dim)
            #only select K best from the train set.
            kbest.fit(X[train], y[train])
            X_new = kbest.transform(X)   
            
        X_new = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))
        
        X_train, X_test = X_new[train], X_new[test]
        y_train, y_test = y[train], y[test]
        
        #Need a validation set. Take 15% of the train data.
        x_tr, x_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.15, stratify=y_train)
        
        y_tr = ohe.transform(y_tr.reshape(-1,1))
        y_val = ohe.transform(y_val.reshape(-1,1))
        y_test = ohe.transform(y_test.reshape(-1,1))
        
        #Add gaussian noise. I've found this improves results.
        # Instead of guessing sigma, get it from X_train.
        sigma = X_train.flatten().std() * sigma_scale
        x_tr, y_tr = gaussian_expansion(x_tr, y_tr, noise_size, sigma, 128)
        
        #Create model
        model = create_model(emb_dim)
        
        #Fit model, using validation set for early stopping
        model.fit(x_tr, y_tr, batch_size=6, epochs=200, validation_data=(x_val, y_val),
                  callbacks=[EarlyStopping('val_loss', patience=35, restore_best_weights=True)])

        evals = model.evaluate(X_test, y_test)
        clear_output()
        scores.append(evals[1])
    return {'dataset': dataset_name,
            'feature_repr': processing_name,
            'n_features': emb_dim,
            'noise_sigma': sigma,
            'noise_magnitude': noise_size,
            'cv_n_fold': n_folds,
            'mean_acc': np.mean(scores),
            'mean_std': np.std(scores)}

In [ ]:
def kfold_cnn(X, y, 
              n_folds, 
              sigma_scale, noise_size, 
              emb_dim, 
              dataset_name,
              processing_name,
              otu_table=False):

In [6]:
#'cdi_youngster' removed since it apparenlty is not balanced at all.
studies = ['asd_son',  'crc_baxter', 'crc_zeller', 'hiv_lozupone', 'hiv_noguerajulian',
          'ibd_gevers_2014', 'ob_goodrich', 't1d_alkanani']

In [13]:
###########################
K = [6,8]
cv_fold = 2
embed_dim = 128
##########################

noise_params = [(1, 0),
                (1, 10),
                (1, 100),
                (1, 500),
                (10, 10),
                (10, 100),
                (10, 500),
                (100, 10),
                (100, 100),
                (100, 500)]


all_results = []
for study in studies:
    df = clean_table(study)
    #otu trials
    otu_desc = 'otu_relative_abundance'
    X, y = relative_abundance_xy(df)
    for params in noise_params:
        all_results.append(
            kfold_cnn(X, y, cv_fold, params[0], params[1], embed_dim, study, otu_desc, otu_table=True)
        )
    #kmer and BPE
    for k in K:
        #kmers first
        kmer_desc = 'kmer_size_{}_document_vectors'.format(k)
        d2v = Doc2Vec.load('doc2vecmodels/doc2vec_{}mers_128dim.model'.format(k))
        
        X, y = kmer_docs_xy(df, d2v, k)
        for params in noise_params:
            all_results.append(
                kfold_cnn(X, y, cv_fold, params[0], params[1], embed_dim, study, kmer_desc, otu_table=False)
            )
        
        #bpe 
        bpe_desc = 'bpe_size_{}_document_vectors'.format(k)
        d2v = Doc2Vec.load('doc2vecmodels/doc2vec_{}bpe_128dim.model'.format(k))
        #sentencpiece
        SP_FILE = 'bpe_models/{}mer_compare_bpe_dna_wordsize_256.model'.format(k)
        sp = spm.SentencePieceProcessor()
        sp.Load(SP_FILE)
        
        X, y = bpe_docs_xy(df, sp, d2v)
        for params in noise_params:
            all_results.append(
                kfold_cnn(X, y, cv_fold, params[0], params[1], embed_dim, study, bpe_desc, otu_table=False)
            )

del(X)
del(y)
df = pd.DataFrame.from_records(all_results)
df.to_csv('experiment_results.tsv', sep='\t', index=False)